In [156]:
import findspark
findspark.init('/home/emanalytics/spark-3.3.1-bin-hadoop3')

import pyspark 
import os 
import glob

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

def create_spark_session(): 
    spark = SparkSession.builder.appName('sparkify').getOrCreate()
    return spark 

In [2]:
spark=create_spark_session() 

22/12/30 10:05:57 WARN Utils: Your hostname, emanalytics-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/30 10:05:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/30 10:06:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [33]:
df = spark.read.json('data/CA_category_id.json', multiLine=True)
df.printSchema()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)



In [25]:
df.select('items')

DataFrame[items: array<struct<etag:string,id:string,kind:string,snippet:struct<assignable:boolean,channelId:string,title:string>>>]

In [34]:
df1 = df.withColumn('data', f.explode('items')).select('data')

In [35]:
df1.withColumn('id', f.col("data.id")) \
    .withColumn('title', f.col("data.snippet.title")).select('id', 'title').count()

31

In [137]:
dir = glob.glob('data/*.json')

schema = StructType([
    StructField('id', IntegerType(), True), 
    StructField('title', StringType(), True), 
    StructField('region', StringType(), True)
])

final = spark.createDataFrame([], schema)
for fn in dir:
    print(fn)
    a = fn.split('/')
    b = a[1].split('_')
    print(b[0])
    df = spark.read.format('org.apache.spark.sql.json').option('multiline', 'true').load(fn)
    df1 = df.withColumn('data', f.explode('items')).select('data').dropDuplicates()
    df2 = df1.withColumn('id', f.col("data.id")) \
                .withColumn('title', f.col("data.snippet.title")) \
                .withColumn('region', f.lit(b[0])).select('id', 'title', 'region').dropDuplicates()
    final = final.union(df2)
    

data/CA_category_id.json
CA
data/JP_category_id.json
JP
data/MX_category_id.json
MX
data/DE_category_id.json
DE
data/US_category_id.json
US
data/RU_category_id.json
RU
data/GB_category_id.json
GB
data/IN_category_id.json
IN
data/KR_category_id.json
KR
data/FR_category_id.json
FR


In [138]:
#final.filter(final['region']=='CA').sort('id').show()
final.groupBy('region').count().show()

+------+-----+
|region|count|
+------+-----+
|    CA|   31|
|    JP|   31|
|    MX|   31|
|    DE|   31|
|    US|   32|
|    RU|   31|
|    GB|   31|
|    IN|   31|
|    KR|   31|
|    FR|   31|
+------+-----+



In [55]:
df1 = df.withColumn('data', f.explode('items')).select('data')


In [58]:
df2 = df1.withColumn('id', f.col("data.id")) \
    .withColumn('title', f.col("data.snippet.title")) \
    .withColumn('filename', f.lit('Something')).select('id', 'title', 'filename')
df2.show()

+---+--------------------+---------+
| id|               title| filename|
+---+--------------------+---------+
|  1|    Film & Animation|Something|
|  2|    Autos & Vehicles|Something|
| 10|               Music|Something|
| 15|      Pets & Animals|Something|
| 17|              Sports|Something|
| 18|        Short Movies|Something|
| 19|     Travel & Events|Something|
| 20|              Gaming|Something|
| 21|       Videoblogging|Something|
| 22|      People & Blogs|Something|
| 23|              Comedy|Something|
| 24|       Entertainment|Something|
| 25|     News & Politics|Something|
| 26|       Howto & Style|Something|
| 27|           Education|Something|
| 28|Science & Technology|Something|
| 30|              Movies|Something|
| 31|     Anime/Animation|Something|
| 32|    Action/Adventure|Something|
| 33|            Classics|Something|
+---+--------------------+---------+
only showing top 20 rows



In [126]:
dir = glob.glob('data/*.json')

schema = StructType([
    StructField('id', IntegerType(), True), 
    StructField('title', StringType(), True), 
    StructField('region', StringType(), True)
])

final = spark.createDataFrame([], schema)
for fn in dir:
    print(fn)
    #print(file)
    a = fn.split('/')
    b = a[1].split('_')
    print(b[0])

data/CA_category_id.json
CA
data/JP_category_id.json
JP
data/MX_category_id.json
MX
data/DE_category_id.json
DE
data/US_category_id.json
US
data/RU_category_id.json
RU
data/GB_category_id.json
GB
data/IN_category_id.json
IN
data/KR_category_id.json
KR
data/FR_category_id.json
FR


In [132]:
df = spark.createDataFrame([], StructType([]))
df.show()

++
||
++
++



In [170]:
dir = glob.glob('data/landing/*.csv')

schema = StructType([
    StructField('video_id', IntegerType(), True), 
    StructField('trending_date', DateType(), True), 
    StructField('title', StringType(), True),
    StructField('channel_title', IntegerType(), True), 
    StructField('category_id', IntegerType(), True), 
    StructField('tags', IntegerType(), True), 
    StructField('views', IntegerType(), True), 
    StructField('likes', IntegerType(), True),
    StructField('dislikes', IntegerType(), True), 
    StructField('comment_count', IntegerType(), True), 
    StructField('region', StringType(), True)
 
])

ytfinal = spark.createDataFrame([], schema)

for file in dir: 
    #print(file)
    a = file.split('/')
    b = a[2]
    print(b[0:2])

    df = spark.read.csv(file, header=True, inferSchema=True)

    df2 = df.select('video_id', \
    'trending_date',  \
    'title', \
    'channel_title', \
    'category_id', \
    'tags', \
    'views', \
    'likes', \
    'dislikes', \
    'comment_count')

    df3 = df2.withColumn('region', f.lit(b[0:2]))


    ytfinal = ytfinal.union(df3)
    #ytfinal = ytfinal.withColumn('region', f.lit(b[0:2]))

GB


IN


US


JP


FR


CA


MX


KR


RU


DE


In [171]:
ytfinal.groupBy('region').count().show()

+------+-----+
|region|count|
+------+-----+
|    GB|43295|
|    IN|38528|
|    US|48137|
|    JP|21445|
|    FR|46138|
|    CA|45560|
|    MX|43819|
|    KR|36730|
|    RU|46260|
|    DE|46957|
+------+-----+



In [163]:
ytfinal.show()

schema = StructType([
    StructField('video_id', IntegerType(), True), 
    StructField('trending_date', DateType(), True), 
    StructField('title', StringType(), True),
    StructField('channel_title', IntegerType(), True), 
    StructField('category_id', IntegerType(), True), 
    StructField('tags', IntegerType(), True), 
    StructField('views', IntegerType(), True), 
    StructField('likes', IntegerType(), True),
    StructField('dislikes', IntegerType(), True), 
    StructField('comment_count', IntegerType(), True), 
 
])

print(hds)


+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------+-------+--------+-------------+
|   video_id|trending_date|               title|       channel_title|category_id|                tags|   views|  likes|dislikes|comment_count|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------+-------+--------+-------------+
|Jw1Y-zhQURU|     17.14.11|John Lewis Christ...|          John Lewis|         26|"christmas"|"john...| 7224515|  55681|   10247|         9479|
|3s1rvMFUweQ|     17.14.11|Taylor Swift: …Re...| Saturday Night Live|         24|"SNL"|"Saturday N...| 1053632|  25561|    2294|         2757|
|n1WpP7iowLc|     17.14.11|Eminem - Walk On ...|          EminemVEVO|         10|"Eminem"|"Walk"|"...|17158579| 787420|   43420|       125882|
|PUTEiSjKwJU|     17.14.11|Goals from Salfor...|Salford City Foot...|         17|"Salford City FC"...|   27833|    193|      12|           37|

In [160]:
df.select('video_id', \
    'trending_date',  \
    'title', \
    'channel_title', \
    'category_id', \
    'tags', \
    'views', \
    'likes', \
    'dislikes', \
    'comment_count').show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------+------+--------+-------------+
|   video_id|trending_date|               title|       channel_title|category_id|                tags|   views| likes|dislikes|comment_count|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------+------+--------+-------------+
|kzwfHumJyYc|     17.14.11|Sharry Mann: Cute...|     Lokdhun Punjabi|          1|"sharry mann|""sh...| 1096327| 33966|     798|          882|
|zUZ1z7FwLc8|     17.14.11|पीरियड्स के समय, ...|             HJ NEWS|         25|"पीरियड्स के समय|...|  590101|   735|     904|            0|
|10L1hZ9qa58|     17.14.11|Stylish Star Allu...|                TFPC|         24|"Stylish Star All...|  473988|  2011|     243|          149|
|N1vE8iiEg64|     17.14.11|Eruma Saani | Tam...|         Eruma Saani|         23|"Eruma Saani|""Ta...| 1242680| 70353|    1624|         2684|
|kJzGH